# Training InceptionResnetV1 as Encoder

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bef05a4b-cb97-16b4-782b-e4e660fabff0)


#### Mount the Drive and navigate to the project directory
Only relevent if you are running in colab.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/MastersDegree/Semester3/046211_project/046211_project_repo"

/content/drive/.shortcut-targets-by-id/16Sgobjo7iyQGdDH7yO6fW1KrvK_Ol-8K/046211_project/046211_project_repo


#### Add all the imports

In [4]:
import os
import time

import numpy as np
import torch
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import dnnlib
import legacy
from facenet_encoder import training
from facenet_encoder.inception_resnet_v1 import InceptionResnetV1
from facenet_encoder.utils import fixed_image_standardization, one_hot_vector

#### Import a Generator to use in the training

In [5]:
def import_generator(generator_pkl):
    """
    Create a dataset of generated images and their corresponding latent vectors using a trained generator.
    Args:
        generator_pkl:  Path to the pkl file from which to load the generator.

    Returns: Instance of teh generator loaded from the pkl file.
    """
    g_kwargs = dnnlib.EasyDict()
    g_kwargs.size = None
    g_kwargs.scale_type = 'pad'
    with dnnlib.util.open_url(generator_pkl) as f:
        generator = legacy.load_network_pkl(f, custom=True, **g_kwargs)['G_ema']
    return generator


#### Define a Dataset for creating examples using a generator

In [6]:
class GenerativeDataset(Dataset):
    def __init__(self, generator, transform, length):
        self.generator = generator
        self.transform = transform
        self.length = length
        self.z_size = generator.z_dim
        self.n_classes = generator.c_dim

    def __len__(self):
        return self.length

    def __getitem__(self, item):
        self.generator.eval()
        with torch.no_grad():
            random_z = torch.randn([1, self.z_size])
            class_idx = np.random.randint(0, self.n_classes)
            class_vector = one_hot_vector(self.n_classes, class_idx)
            gen_img = self.generator(random_z, class_vector).squeeze()
            gen_img = self.transform(gen_img)
        return gen_img.cpu().numpy(), random_z.squeeze().cpu().numpy()

#### Perform the Training

Determine if an nvidia GPU is available

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


Define run parameters. Modify `generator_pkl` to the location of the pkl file of the generator you wish to use when training.

In [11]:
generator_pkl = '../out/training-runs/network-snapshot-000900.pkl'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 2
dataset_len = 10_000

Load the generator

In [9]:
generator = import_generator(generator_pkl)
print('Loaded generator model from {}'.format(generator_pkl))

Loaded generator model from ../out/training-runs/network-snapshot-000900.pkl


Create a new GenerativeDataset, then create a Tringing Loader and a Validation Loader.

In [12]:
trans = transforms.Compose([
    transforms.ConvertImageDtype(torch.float32),
    fixed_image_standardization
])

dataset = GenerativeDataset(generator, trans, dataset_len)
print(f'Created dataset of length {dataset_len}.')

img_inds = np.arange(dataset_len)
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]


train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
print(f'Created train loader of size {len(train_loader)} batches.')
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)
print(f'Created validation loader of size {len(val_loader)} batches.')

Creating train loader...
Creating validation loader...


Define Inception Resnet V1 model.

See `help(InceptionResnetV1)` for more details.

In [13]:
encoder = InceptionResnetV1(pretrained='vggface2', device=device)

Define optimizer, scheduler, loss function and training metrics

In [14]:
optimizer = optim.Adam(encoder.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

loss_fn = torch.nn.MSELoss()
metrics = {
    'fps': training.BatchTimer(),
}

In [15]:
training_path = os.path.join('..', 'out', 'encoder_trainings')
os.makedirs(training_path, exist_ok=True)

#### Train model

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 20)
start = time.time()
encoder.eval()
training.pass_epoch(encoder, loss_fn, val_loader, batch_metrics=metrics, show_running=True, device=device, writer=writer)
end = time.time()
print('Elapsed time {}'.format(end - start))

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 20)

    start = time.time()
    encoder.train()
    training.pass_epoch(encoder, loss_fn, train_loader, optimizer, scheduler, batch_metrics=metrics,
                        show_running=True, device=device, writer=writer)

    encoder.eval()
    training.pass_epoch(encoder, loss_fn, val_loader, batch_metrics=metrics, show_running=True, device=device,
                        writer=writer)
    end = time.time()
    print('Elapsed time {}'.format(end - start))

    encoder_name = 'incresnetv1_encoder_epoch{}.pt'.format(epoch)
    file_name = os.path.join(training_path, encoder_name)
    print('\nSaving encoder model to {} ...'.format(file_name))
    torch.save(encoder.state_dict(), file_name)

writer.close()
print('\nFinished training.')



Initial
--------------------
Valid |    63/63   | loss:    1.0048 | fps:   28.4882   
Elapsed time 777.1451733112335

Epoch 1/8
--------------------
Train |   146/250  | loss:    1.0003 | fps:   12.9488   

Enjoy your trained encoder!